In [10]:
import h5py
import numpy as np

file_path = "NP_002 Tracking/kittracking001-kitjobset_241126_Cell033_withSisterAlignment-Cell033_DC_DS.mat"

with h5py.File(file_path, 'r') as f:
    # Dereference the first entry in dataStruct
    data_struct = f["dataStruct"][0, 0]  # Assuming first struct is relevant
    
    if isinstance(data_struct, h5py.Reference):
        struct_group = f[data_struct]  # Dereference dataStruct
        
        # Navigate to initCoord
        if "initCoord" in struct_group:
            init_coord_group = struct_group["initCoord"]
            
            # Check if allCoordPix exists
            if "allCoordPix" in init_coord_group:
                allCoordPix = init_coord_group["allCoordPix"]
                
                # Read the first entry
                first_entry_ref = allCoordPix[0, 0]  # Assuming shape (127,1)
                
                if isinstance(first_entry_ref, h5py.Reference):
                    first_data = f[first_entry_ref]  # Dereference
                    print(f"First entry shape: {first_data.shape}, Type: {first_data.dtype}")
                    
                    # Load into a NumPy array
                    array_data = first_data[...]
                    print("First entry array:", array_data)
                else:
                    print("First entry is not a reference.")
            else:
                print("allCoordPix not found in initCoord.")
        


First entry shape: (6, 125), Type: float64
First entry array: [[1.90319498e+02 1.87193000e+02 9.69577836e+01 1.70528009e+02
  1.84310571e+02 1.86277988e+02 1.00591277e+02 1.49896850e+01
  9.27543845e+01 2.09940131e+02 1.47028507e+02 1.74949110e+02
  1.66688471e+02 1.92664971e+02 1.89864170e+02 1.88305990e+02
  1.46052333e+02 1.23181265e+02 1.68027027e+02 1.69285499e+02
  9.94959257e+01 1.76270384e+02 1.77792154e+02 2.77929026e+01
  1.61273044e+02 1.61690959e+02 1.01036607e+02 1.73369328e+02
  1.72907666e+02 1.69238303e+02 1.51877291e+02 1.47255338e+02
  1.93534596e+02 1.26318828e+02 1.69470754e+02 1.52652051e+02
  1.55139902e+02 2.15828139e+02 2.04383418e+02 1.30165171e+02
  1.54287940e+02 1.59563619e+02 1.59812538e+02 1.75574082e+02
  1.74929403e+02 1.75117364e+02 1.55172658e+02 1.56088806e+02
  1.40007416e+02 1.63913241e+02 1.63065954e+02 1.64923075e+02
  1.62177385e+02 1.72394196e+02 1.30862675e+02 1.27664946e+02
  1.29434746e+02 1.48209432e+02 1.38800572e+02 1.37803449e+02
  1.3782

In [14]:
ls NP_002\ Tracking

diags_kitjobset_241126_Cell033_withSisterAlignment.txt
diags_kitjobset_241126_Cell037_withSisterAlignment.txt
kitjobset_241126_Cell033_withSisterAlignment.mat
kitjobset_241126_Cell037_withSisterAlignment.mat
kittracking001-kitjobset_241126_Cell033_withSisterAlignment-Cell033_DC_DS.mat
kittracking001-kitjobset_241126_Cell037_withSisterAlignment-Cell037_DC_DS.mat


In [2]:
pip install --upgrade scikit-image

Note: you may need to restart the kernel to use updated packages.
